# CNN / Reconstructing Autoencoder on MNIST data with TensorFlow 2

Sources for this notebook
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://colab.research.google.com/github/ageron/handson-ml2/blob/master/17_autoencoders_and_gans.ipynb

Latent Representation learns abstract concepts about inputs. We show two use cases
1. Semantic dimensionality reduction on images: using latent representation
2. modification of images: example is denoising

A big thanks to [Oliver Zeigermann](https://github.com/DJCordhose)

## Setup

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

dpi = 96
# dpi = 300

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15, 8)
# mpl.rcParams["figure.dpi"] = dpi
mpl.rc('xtick', labelsize=15) 
mpl.rc('ytick', labelsize=15)

In [0]:
# Gives us a well defined version of tensorflow

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
tf.random.set_seed(42)

In [0]:
!nvidia-smi

In [0]:
import numpy as np
np.random.seed(42)

In [0]:
# for RAdam
!pip install -q tensorflow_addons

In [0]:
import tensorflow_addons as tfa

In [0]:
# https://github.com/AndreasMadsen/python-lrcurve
!pip install -q lrcurve 

In [0]:
from lrcurve import KerasLearningCurve

## Fashion MNIST data set

In [0]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

x_res = 28
y_res = 28
image_size = x_res * y_res

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train.shape, x_test.shape

In [0]:
def plot_samples(x, y, n_samples=15, labels=None):
  figure = plt.figure()
  for i, index in enumerate(np.random.choice(x_test.shape[0], size=n_samples, replace=False)):
      ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
      ax.imshow(x[index].reshape(x_res, y_res), cmap="binary")
      if labels:
        ax.set_title("{}".format(labels[y[index]]))
      else:
        ax.set_title("{}".format(y[index]))

In [0]:
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

np.random.seed(42)
plot_samples(x_train, y_train, labels=fashion_mnist_labels)

## Autoencoder - CNN with Dense Lantent Space

In [0]:
from tensorflow import keras

@tf.function
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [0]:
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Reshape
from tensorflow.keras.models import Model, Sequential

encoding_dim = 128

tf.random.set_seed(42)

conv_encoder = Sequential([
    Input(shape=(x_res, y_res)),
    Reshape((x_res, y_res, 1)),
    Conv2D(16, kernel_size=3, padding="same", activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(encoding_dim, activation="relu", 
          activity_regularizer=keras.regularizers.l2(1e-4)
         ),

])
conv_decoder = Sequential([
    Input(shape=(encoding_dim)),
    Reshape((1, 1, encoding_dim)),
    Conv2DTranspose(32, kernel_size=3, strides=2, padding="valid", activation="relu"),
    Conv2DTranspose(16, kernel_size=3, strides=2, padding="valid", activation="relu"),
    Conv2DTranspose(8, kernel_size=3, strides=2, padding="same", activation="relu"),
    Conv2DTranspose(1, kernel_size=3, strides=2, padding="same", activation="sigmoid"),
    Reshape((x_res, y_res))
])
conv_ae = Sequential([conv_encoder, conv_decoder])

# Pick your optimizer
momentum_with_nesterov = tf.optimizers.SGD(momentum=0.95,nesterov=True)
adam_vanilla = tf.optimizers.Adam()
adam_amsgrad = tf.optimizers.Adam(amsgrad=True)
radam = tfa.optimizers.RectifiedAdam()
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

#conv_ae.compile(loss="binary_crossentropy", optimizer='SGD', metrics=[rounded_accuracy])
#conv_ae.compile(loss="binary_crossentropy", optimizer=momentum_with_nesterov, metrics=[rounded_accuracy])
#conv_ae.compile(loss="binary_crossentropy", optimizer=adam_vanilla, metrics=[rounded_accuracy])
#conv_ae.compile(loss="binary_crossentropy", optimizer=adam_amsgrad, metrics=[rounded_accuracy])
#conv_ae.compile(loss="binary_crossentropy", optimizer=radam, metrics=[rounded_accuracy])
#conv_ae.compile(loss="binary_crossentropy", optimizer=ranger, metrics=[rounded_accuracy])

conv_encoder.summary()
conv_decoder.summary()

In [0]:
%%time

BATCH_SIZE = 256
#BATCH_SIZE = 64
EPOCHS = 20 # Try 50/100/...

history = conv_ae.fit(x = x_train,
                    y = x_train,
                    validation_data=(x_test, x_test),
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    callbacks=[KerasLearningCurve()],
                    verbose=0)

In [0]:
START_EPOCHE = 0
END_EPOCHE = -1

plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.title('Accuracy over time')

plt.plot(history.history['rounded_accuracy'][START_EPOCHE:END_EPOCHE])
plt.plot(history.history['val_rounded_accuracy'][START_EPOCHE:END_EPOCHE])

plt.legend(['Training', 'Test']);

In [0]:
x_test_pred = conv_ae.predict(x_test)
np.random.seed(42)
plot_samples(x_test_pred, y_test, labels=fashion_mnist_labels)
# plot_samples(x_test_pred, y_test)

In [0]:
loss, accuracy = conv_ae.evaluate(x_train, x_train, verbose=0)
loss, accuracy

In [0]:
loss, accuracy = conv_ae.evaluate(x_test, x_test, verbose=0)
loss, accuracy

In [0]:
def compare_samples(x, x_pred, n_samples=5):
  figure = plt.figure()
  for i, index in enumerate(np.random.choice(x_test.shape[0], size=n_samples, replace=False)):
      ax = figure.add_subplot(2, n_samples, i + 1, xticks=[], yticks=[])
      ax.imshow(x[index].reshape(x_res, y_res), cmap="binary")
      ax = figure.add_subplot(2, n_samples, i + 1 + n_samples, xticks=[], yticks=[])
      ax.imshow(x_pred[index].reshape(x_res, y_res), cmap="binary")

In [0]:
np.random.seed(42)
compare_samples(x_test, x_test_pred)

## Noisy Images - Feel free to apply the above methods here

In [0]:
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

plot_samples(x_train_noisy, y_train, labels=fashion_mnist_labels)

In [0]:
noise = keras.layers.GaussianNoise(0.2)
dropout = keras.layers.Dropout(0.2)

x_train_noisy = dropout(x_train, training=True).numpy()
x_test_noisy = dropout(x_test, training=True).numpy()

plot_samples(x_train_noisy, y_train, labels=fashion_mnist_labels)

In [0]:
noise = keras.layers.GaussianNoise(0.2)

x_train_noisy = noise(x_train, training=True).numpy()
x_test_noisy = noise(x_test, training=True).numpy()

plot_samples(x_train_noisy, y_train, labels=fashion_mnist_labels)

In [0]:
x_train_noisy = tf.image.flip_left_right(x_train).numpy()
x_test_noisy = tf.image.flip_left_right(x_test).numpy()

plot_samples(x_train_noisy, y_train, labels=fashion_mnist_labels)

In [0]:
threshold = 0.5

x_train_noisy = (x_train > threshold).astype(float)
x_test_noisy =  (x_test > threshold).astype(float)

plot_samples(x_train_noisy, y_train, labels=fashion_mnist_labels)

In [0]:
%%time

BATCH_SIZE = 256
EPOCHS = 50

denoising_ae = Sequential([conv_encoder, conv_decoder])
denoising_ae.compile(loss="binary_crossentropy", optimizer='adam', metrics=[rounded_accuracy])

history = denoising_ae.fit(x = x_train_noisy,
                    y = x_train,
                    validation_data=(x_test_noisy, x_test),
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    callbacks=[KerasLearningCurve()],
                    verbose=0)

In [0]:
denoising_ae.evaluate(x = x_train_noisy, y = x_train, batch_size=BATCH_SIZE, verbose=0)

### Noisy denoised

In [0]:
x_test_pred = denoising_ae.predict(x_test_noisy)
plot_samples(x_test_pred, y_test, labels=fashion_mnist_labels)
# plot_samples(x_test_pred, y_test)

### Original "denoised"

In [0]:
x_test_pred = denoising_ae.predict(x_test)
plot_samples(x_test_pred, y_test, labels=fashion_mnist_labels)
# plot_samples(x_test_pred, y_test)

In [0]:
x_test_pred = denoising_ae.predict(x_test_noisy)

compare_samples(x_test_noisy, x_test_pred)